# 8.29
## T1. [Five Dimensional Points](https://codeforces.com/problemset/problem/850/A)
- **题意**: 给定一个5维空间中的n个点，计算其中“完美点”的个数
- **题解**: 考虑**上限情况**，然后暴力求解，事实上，需要考虑的点个数最多不会超过$2\times k + 1$个点
___
**Proposition**: 给定$k$维空间的$n$个点，构造两两夹角非锐角的向量，最多需要考虑的节点个数不超过$2\times k + 1$

**Proof**: 在$k$维空间中，固定一个点，构造两两间夹角非锐角的向量，个数最多的情况显然是两两正交，此时向量个数最多为$k$(线性空间的维度)，考虑正负两个方向，此时需要计算的点个数为$2\times k$，再考虑出发点，最终需要考虑的点个数最多为$2 \times k + 1$，当节点个数超过此值时，必然会出现两个向量夹角为锐角.

In [ ]:
def vec(a, b):
    return [y - x for x, y in zip(a, b)]

def cal(a, b): 
    return sum([x * y for x, y in zip(a, b)])

n = int(input()) 
if n >= 12: 
    print(0)
    exit()
pts = [list(map(int, input().split())) for _ in range(n)]

outs = [] 

for i in range(n): 
    ok = True 
    for j in range(n): 
        for k in range(j):
            if cal(vec(pts[i], pts[j]), vec(pts[i], pts[k])) > 0: 
                ok = False 
    if ok: 
        outs.append(f"{i + 1}")

print(len(outs))
print(" ".join(outs))

## T2. [Robo-Footballer](https://codeforces.com/problemset/problem/248/C)
- **题意**: 在y轴上有一个区间，表示一个球门，给定球的位置$(x_b, y_b)$，需要通过将球踢到墙($y=y_w$)上，通过折射之后让球通过球门
- **题解**:
  本题主要考察问题转换:  
  1. 考虑球门的镜像区间, 将折射问题简化为求直线交点
  2. 考虑到球的直径$r$, 判断条件转换为判断镜像后区间的下端点与球的线路的最短距离是否超过$r$

In [ ]:
y1, y2, yw, xb, yb, r = map(int, input().split())
y1, y2 = 2 * yw - 2 * r - y2, 2 * yw - 2 * r - y1 

if r ** 2 * ((yb - y2 + r) ** 2 + xb ** 2) > xb ** 2 * (y2 - y1 - r) ** 2: 
    print(-1)
else:
    print((yw - y2) * xb / (yb - y2 + r))

# 8.30
## T1. [Code For 1(CF)](https://codeforces.com/problemset/problem/768/B)
- 题意: 给定一个列表，初始只有一个元素$n$，每次取出一个大于1的元素$x$，并在相同位置加入三个数，分别是$\frac{x}{2}, x \text{mod} 2, \frac{x}{2}$，直至最终全为1
- 题解: 首先问题中的列表可以转换为满二叉树的中序遍历序列，只需要确定每个节点的访问编号即可，由于除完后的每个数必然都是1，只需要判断非叶子节点的奇偶性即可，可以使用递归的判断。

In [ ]:
# 问题中的列表可以转换为满二叉树的中序遍历序列
# 只需要确定每个节点的访问编号即可，由于除完后的每个数必然都是1，只需要用递归判断编号在[l, r]内所有非叶子节点的奇偶性
# https://codeforces.com/contest/768/submission/278711521

n, l, r = map(int, input().split())

ans = r - l + 1

def dfs(v: int, c: int): 
    if v == 0 or v == 1: 
        return 1 - v

    res = 0
    cnt = (1 << (v >> 1).bit_length()) - 1 + c # 可能访问的节点总个数

    if l <= cnt + 1: 
        # 访问左子树
        if cnt < r: 
            # 判断当前节点
            res += 1 - v % 2
        res += dfs(v >> 1, c)
        # 访问右子树
        if cnt + 1 < r: 
            res += dfs(v >> 1, cnt + 1)
    else: 
        # 访问右子树
        res += dfs(v >> 1, cnt + 1)
    
    return res
    
res = dfs(n, 0)
print(ans - res)

## T2. [Destroy it!](https://codeforces.com/problemset/problem/1176/F)
- 题意: n个回合，每个回合给定k张卡片，每次选择代价不超过3的卡片，逢10加倍，计算最大伤害
- 题解: 
  1) 容易联想到背包问题，但是有个选择的卡片"逢10加倍"的条件，遇到这种具有周期性的问题，可以考虑对周期取模构建状态表达:
  - 状态`dp[i]`: 在某一回合选择的卡片个数对10取模为i的最大伤害
  2) 每一回合最大代价不能超过3，因此每一回合的方案个数其实是固定的，只需要暴力枚举所有方案即可

In [ ]:
# https://codeforces.com/contest/1176/submission/278734296

n = int(input()) 
inf = 10 ** 11 
dp  = [0] + [-inf] * 9 
ndp = [-inf] * 10

for _ in range(n): 
    k = int(input())
    vs = [[] for _ in range(4)]

    for _ in range(k):
        i, v = map(int, input().split())
        vs[i].append(v)

    for i in range(1, 4): 
        vs[i] = nlargest(3 // i, vs[i]) 

    for c1 in range(min(3, len(vs[1])) + 1): 
        for c2 in range(min(c1 <= 1, len(vs[2])) + 1):
            for c3 in range(min(c1 == 0 and c2 == 0, len(vs[3])) + 1): 

                d  = sum(vs[1][:c1]) + sum(vs[2][:c2]) + sum(vs[3][:c3])
                mx = max([0] + vs[1][:c1] + vs[2][:c2] + vs[3][:c3])

                for i in range(10): 
                    v = dp[i] + d
                    cnt = i + c1 + c2 + c3 
                    if cnt >= 10:
                        cnt -= 10 
                        v   += mx 
                    ndp[cnt] = max(ndp[cnt], v)
                    
    dp, ndp = ndp[:], [-inf] * 10

print(max(dp))

# 8.31
## T1. [Preparing for Merge Sort](https://codeforces.com/problemset/problem/847/B)
- 题意: 给一个长度为n的序列，按顺序扫描，得到数个单调递增子序列
- 题解: 二分的应用，一道比较典的题，类似于单调栈，记录下每个子序列最后一个数，这个数组一定是单调递减的，这个时候可以用二分寻找每次插入的位置即可

In [ ]:
n = int(input()) 
a = map(int, input().split()) 

outs = [[]] 
lsts = [0] 

for x in a: 
    l, r = 0, len(lsts) - 1
    while l < r: 
        mid = l + r >> 1 
        if lsts[mid] < x: r = mid 
        else: l = mid + 1 
    if lsts[l] >= x: 
        outs.append([f'{x}'])
        lsts.append(x)
    else:
        outs[l].append(f'{x}')
        lsts[l] = x 

print('\n'.join([' '.join(out) for out in outs]))